In [1]:
from google.colab import files
uploaded = files.upload()


Saving tumor_dataset.zip.zip to tumor_dataset.zip.zip
Saving tumor_dataset1.zip.zip to tumor_dataset1.zip.zip


In [3]:
import zipfile
import os

zip_path = "tumor_dataset1.zip"  # Change this if you uploaded another file

# Extract ZIP file
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall("tumor_dataset")

# Verify extracted files
print("Extracted files:", os.listdir("tumor_dataset"))


FileNotFoundError: [Errno 2] No such file or directory: 'tumor_dataset1.zip'

In [ ]:
import os

dataset_path = "tumor_dataset"
print("Dataset folders:", os.listdir(dataset_path))


In [ ]:
# Count images in each category
for category in os.listdir(dataset_path):
    category_path = os.path.join(dataset_path, category)
    print(f"{category}: {len(os.listdir(category_path))} images")


In [ ]:
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import matplotlib.pyplot as plt


In [ ]:
IMG_SIZE = (224, 224)

def load_images(folder, label):
    images, labels = [], []
    for file in os.listdir(folder):
        img_path = os.path.join(folder, file)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, IMG_SIZE) / 255.0  # Normalize images
            images.append(img)
            labels.append(label)
    return images, labels


In [ ]:
# Define folder paths
tumor_folder = os.path.join(dataset_path, "tumor")
normal_folder = os.path.join(dataset_path, "normal")

# Load images
tumor_images, tumor_labels = load_images(tumor_folder, 1)
normal_images, normal_labels = load_images(normal_folder, 0)

# Convert to NumPy arrays
X = np.array(tumor_images + normal_images)
y = np.array(tumor_labels + normal_labels)

# Shuffle dataset
X, y = shuffle(X, y, random_state=42)

# Split into training & testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training samples:", len(X_train))
print("Testing samples:", len(X_test))


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# CNN Model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

# Compile Model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Model Summary
model.summary()


In [ ]:
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


In [ ]:
# Plot training history
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title("Model Accuracy")
plt.show()


In [ ]:
model.save("brain_tumor_model.h5")


In [ ]:
from google.colab import files

# Upload an image for testing
uploaded = files.upload()

# Get filename
image_path = list(uploaded.keys())[0]
print("Uploaded file:", image_path)


In [ ]:
def predict_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (224, 224)) / 255.0
    img = np.expand_dims(img, axis=0)

    prediction = model.predict(img)[0][0]
    return "Tumor Detected" if prediction > 0.5 else "No Tumor"

print(f"Prediction for {image_path}: {predict_image(image_path)}")
